In [1]:
import math
import numpy as np
import sys
from typing import List, Tuple
from functools import lru_cache, reduce, cmp_to_key
from itertools import starmap, accumulate, chain, islice, product, tee, pairwise, combinations
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
from importlib import reload

reload(arrays)
reload(trees)
reload(lists)
reload(graphs)
reload(search)
import arrays as ar
import trees as tr
import lists as ll
import graphs as g
from lists import Node

print()

In [122]:
def longest_prefix_suffix(p: str) -> List[int]:
    n = len(p)
    lps = [0] * n
    i, j = 0, 1
    while j < n:
        if p[i] == p[j]:
            print(f'{p[i]} == {p[j]}: {i}    \t {j} = {i+1}   \t {p[0:i]} == {p[j-i:j]}')
            lps[j] = i = i + 1
        elif i > 0:
            print(f'{p[i]} != {p[j]}: {lps[i-1]}<-{i} \t {j} = {lps[j]}  \t {p[0:i]} == {p[j-i:j]}')
            #j = j - i + 1
            i = lps[i-1]
            continue
        j += 1
    print(lps)
    return lps[-1]


In [132]:
longest_prefix_suffix("abccc0aaabccc0aabc")

a == a: 0    	 6 = 1   	  == 
b != a: 0<-1 	 7 = 0  	 a == a
a == a: 0    	 7 = 1   	  == 
b != a: 0<-1 	 8 = 0  	 a == a
a == a: 0    	 8 = 1   	  == 
b == b: 1    	 9 = 2   	 a == a
c == c: 2    	 10 = 3   	 ab == ab
c == c: 3    	 11 = 4   	 abc == abc
c == c: 4    	 12 = 5   	 abcc == abcc
0 == 0: 5    	 13 = 6   	 abccc == abccc
a == a: 6    	 14 = 7   	 abccc0 == abccc0
a == a: 7    	 15 = 8   	 abccc0a == abccc0a
a != b: 1<-8 	 16 = 0  	 abccc0aa == abccc0aa
b == b: 1    	 16 = 2   	 a == a
c == c: 2    	 17 = 3   	 ab == ab
[0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 3, 4, 5, 6, 7, 8, 2, 3]


3